<a href="https://colab.research.google.com/github/cam2149/MachineLearningV/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Equipo**

- Nicolás Colmenares

- Carlos Martinez

1. Análisis Exploratorio de Datos (EDA) - 0.5 pts

  -  Carga y limpieza de los datos.

  - Visualización de tendencias y patrones de dengue.

  -  Análisis de correlaciones entre variables.

**Situación:**
Una ciudad enfrenta un aumento significativo de casos de dengue, con una tasa de incidencia que supera el promedio nacional.
La anticipación de brotes es crucial para implementar medidas preventivas y reducir la propagación de la enfermedad.

**Objetivo:**
Desarrollar un modelo predictivo utilizando redes neuronales para pronosticar futuros brotes de dengue en cada barrio de la ciudad.
Utilizar una base de datos histórica de casos de dengue desde 2015 hasta 2022 para entrenar el modelo.
Anticiparse a los brotes con al menos 3 semanas de anticipación.

**Finalidad:**
Permitir a las autoridades de salud pública tomar acciones oportunas, como:
Preparar a las instituciones prestadoras de salud (IPS).
Gestionar recursos (carros fumigadores, limpieza de sumideros).
Capacitar a la comunidad.

*   Análisis exploratorio de datos de la serie temporal
*   Explicar el comportamiento de la serie temporal en términos cualitativos y cuantitativos para desarrollar intuición para la selección del modelo
*    Identificar los modelos candidatos y los posibles parámetros del modelo que se pueden utilizar basándose en los hallazgos del análisis exploratorio de datos

# 0. Configuraciones de Colab

Mover Kaggle.json a la ubicación correcta después de subirlo

In [ ]:
#Estas líneas son comandos de shell que se ejecutan dentro del Jupyter notebook. Se usan para configurar las credenciales de la API de Kaggle, que son necesarias para descargar conjuntos de datos (datasets) desde Kaggle.

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!rm -rf /content/kaggle/output
!rm -rf /content/kaggle/input

Descargar dataset de la competencia

In [ ]:
!kaggle competitions download -c aa-v-2025-i-pronosticos-nn-rnn-cnn

In [ ]:
!mkdir -p /content/kaggle/output
!mkdir -p /content/kaggle/input

In [ ]:
!mv aa-v-2025-i-pronosticos-nn-rnn-cnn.zip /content/kaggle/input

In [ ]:
!unzip /content/kaggle/input/aa-v-2025-i-pronosticos-nn-rnn-cnn.zip -d /content/kaggle/input/

In [ ]:
#/kaggle/input
import os
for dirname, _, filenames in os.walk('/content/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# 1. Imports

In [ ]:
!pip install altair

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import altair as alt
import matplotlib as mpl
import matplotlib.pyplot as plt   # data visualization
import seaborn as sns
import statsmodels.api as sm
import scipy
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.stats import anderson
from matplotlib.colors import LinearSegmentedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, mean_absolute_error, recall_score, roc_auc_score, accuracy_score, roc_curve



In [ ]:
#Printing library versions
print('Pandas:', pd.__version__)
print('Statsmodels:', sm.__version__)
print('Scipy:', scipy.__version__)
print('Matplotlib:', mpl.__version__)
print('Seaborn:', sns.__version__)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
np.random.seed(786)

# 2. Configs

In [ ]:
config = {
    "TRAIN_DIR": '/content/kaggle/input/df_train.parquet',
    "TEST_DIR": '/content/kaggle/input/df_test.parquet',
    "SUBMISSION_DIR": '/content/sample_submission.csv'
}

# Exploración

## Diccionario

train.parquet - El conjunto de datos de entrenamiento
test.parquet - El conjunto de datos de prueba
sample_submission.csv - un ejemplo de un archivo a someter en la competencia

| **Variable**         | **Descripción**                                                                                      |
|-----------------------|------------------------------------------------------------------------------------------------------|
| id_bar               | identificador único del barrio                                                                      |
| anio                 | Año de ocurrencia                                                                                   |
| semana               | Semana de ocurrencia                                                                               |
| Estrato              | Estrato socioeconómico del barrio                                                                   |
| area_barrio          | Área del barrio en km²                                                                             |
| dengue               | Conteo de casos de dengue                                                                          |
| concentraciones      | Cantidad de visitas e intervención a lugares de concentración humana (Instituciones)                |
| vivienda             | Conteo de las visitas a viviendas a revisión y control de criaderos                                 |
| equipesado           | Conteo de las fumigaciones con Maquinaria Pesada                                                   |
| sumideros            | Conteo de las intervenciones a los sumideros                                                       |
| maquina              | Conteo de las fumigaciones con motomochila                                                         |
| lluvia_mean          | Lluvia promedio en la semana i                                                                     |
| lluvia_var           | Varianza de la lluvia en la semana i                                                               |
| lluvia_max           | Lluvia máxima en la semana i                                                                       |
| lluvia_min           | Lluvia mínima en la semana i                                                                       |
| temperatura_mean     | Temperatura promedio en la semana i                                                                |
| temperatura_var      | Varianza de la temperatura en la semana i                                                          |
| temperatura_max      | Temperatura máxima en la semana i                                                                  |
| temperatura_min      | Temperatura mínima en la semana i                                                                  |


## Lectura del dataset de entrenamiento

In [ ]:
#Esta celda tiene como objetivo leer los datos de entrenamiento desde un archivo Parquet y mostrar información básica sobre ellos
try:
    train_df = pd.read_parquet(config["TRAIN_DIR"]).iloc[:,1:]
    print(train_df.describe())
    print(train_df.info())
    print(train_df.shape)
except FileNotFoundError:
    print("Error: 'series_train.parquet' not found. Please make sure the file exists in the current directory or provide the correct path.")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:

# Función para preparar los datos
def preparar_datos(training_df, target_variable, test_size=0.2, batch_size=32):
    X = training_df.drop(columns=[target_variable]).values
    y = training_df[target_variable].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [ ]:
# Definir el modelo MLP
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Función para entrenar el modelo
def entrenar_modelo(model, train_loader, criterion, optimizer, epochs=100):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}')

In [ ]:
# Función para evaluar el modelo
def evaluar_modelo(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item() * inputs.size(0)

    test_loss /= len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}')
    return test_loss

In [ ]:
# Función para generar el resumen del modelo
def generar_resumen(model, train_loader, test_loader, criterion):
    model.eval()
    train_loss = 0.0
    test_loss = 0.0
    train_outputs = []
    train_targets = []
    test_outputs = []
    test_targets = []

    with torch.no_grad():
        for inputs, targets in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            train_loss += loss.item() * inputs.size(0)
            train_outputs.extend(outputs.view(-1).tolist())
            train_targets.extend(targets.view(-1).tolist())

        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item() * inputs.size(0)
            test_outputs.extend(outputs.view(-1).tolist())
            test_targets.extend(targets.view(-1).tolist())

    train_loss /= len(train_loader.dataset)
    test_loss /= len(test_loader.dataset)

    print(f'Train Loss: {train_loss:.4f}')
    print(f'Test Loss: {test_loss:.4f}')

    # Métricas adicionales
    train_outputs = torch.tensor(train_outputs)
    train_targets = torch.tensor(train_targets)
    test_outputs = torch.tensor(test_outputs)
    test_targets = torch.tensor(test_targets)

    train_mae = mean_absolute_error(train_targets, train_outputs)
    test_mae = mean_absolute_error(test_targets, test_outputs)
    train_mse = mean_squared_error(train_targets, train_outputs)
    test_mse = mean_squared_error(test_targets, test_outputs)
    train_rmse = torch.sqrt(torch.tensor(train_mse)).item()
    test_rmse = torch.sqrt(torch.tensor(test_mse)).item()
    train_accuracy = accuracy_score((train_outputs > 0.5).int(), (train_targets > 0.5).int())
    test_accuracy = accuracy_score((test_outputs > 0.5).int(), (test_targets > 0.5).int())
    train_recall = recall_score((train_outputs > 0.5).int(), (train_targets > 0.5).int())
    test_recall = recall_score((test_outputs > 0.5).int(), (test_targets > 0.5).int())
    train_roc_auc = roc_auc_score((train_targets > 0.5).int(), train_outputs)
    test_roc_auc = roc_auc_score((test_targets > 0.5).int(), test_outputs)

    print(f'Train MAE: {train_mae:.4f}')
    print(f'Test MAE: {test_mae:.4f}')
    print(f'Train MSE: {train_mse:.4f}')
    print(f'Test MSE: {test_mse:.4f}')
    print(f'Train RMSE: {train_rmse:.4f}')
    print(f'Test RMSE: {test_rmse:.4f}')
    print(f'Train Accuracy: {train_accuracy:.4f}')
    print(f'Test Accuracy: {test_accuracy:.4f}')
    print(f'Train Recall: {train_recall:.4f}')
    print(f'Test Recall: {test_recall:.4f}')
    print(f'Train ROC AUC: {train_roc_auc:.4f}')
    print(f'Test ROC AUC: {test_roc_auc:.4f}')

    num_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'Number of trainable parameters: {num_parameters}')

    print(model)

In [ ]:
# Ejecutar todo el pipeline
def ejecutar_mlp(training_df, target_variable, epochs=100, batch_size=32, learning_rate=0.01):
    train_loader, test_loader = preparar_datos(training_df, target_variable, batch_size=batch_size)
    input_dim = training_df.drop(columns=[target_variable]).shape[1]
    model = MLP(input_dim)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    entrenar_modelo(model, train_loader, criterion, optimizer, epochs)
    generar_resumen(model, train_loader, test_loader, criterion)
    test_loss = evaluar_modelo(model, test_loader, criterion)
    return model, test_loss

In [ ]:
# Función para ejecutar el modelo con diferentes combinaciones de hyperparámetros
def ejecutar_experimentos(training_df, target_variable, epochs_list, learning_rates):
    train_loader, test_loader = preparar_datos(training_df, target_variable)
    input_dim = training_df.drop(columns=[target_variable]).shape[1]
    best_loss = float('inf')
    best_params = None
    best_model = None

    for epochs in epochs_list:
        for lr in learning_rates:
            print(f'\nEntrenando modelo con {epochs} epochs y learning rate de {lr}')
            model = MLP(input_dim)
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)
            entrenar_modelo(model, train_loader, criterion, optimizer, epochs)
            test_loss = evaluar_modelo(model, test_loader, criterion)
            if test_loss < best_loss:
                best_loss = test_loss
                best_params = (epochs, lr)
                best_model = model

    print(f'\nMejor modelo: {best_params[0]} epochs, learning rate de {best_params[1]}')
    print(f'Mejor Test Loss: {best_loss:.4f}')
    generar_resumen(best_model, train_loader, test_loader, nn.MSELoss())
    return best_model, best_loss, best_params

In [ ]:
# Definir listas de hyperparámetros
epochs_list = [100, 300, 500, 1000]
learning_rates = [0.01, 0.001]

# Ejemplo de uso
best_model, best_loss, best_params = ejecutar_experimentos(train_df, target_variable='dengue', epochs_list=epochs_list, learning_rates=learning_rates)